# 环境配置

##安装Pytorch

In [1]:
!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


##安装ONNX

In [2]:
# 在进行模型部署时，M个不同的模型需要配置于N个不同的硬件环境上，此时将有M * N种可能，
# 而如果安装了ONNX，则M个不同的模型可以通过ONNX转为统一的文件形式，进而匹配不同的硬件环境，此时就有M + N种可能
# 显然大幅降低了整个部署过程的复杂度

!pip install onnx -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 安装其它第三方工具包

In [4]:
!pip install numpy pandas matplotlib tqdm opencv-python pillow -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


# 基于ImageNet1000类预训练图像分类模型转ONNX

将ImageNet预训练图像分类模型导出为ONNX格式，用于后续在推理引擎上部署。

## 库

In [5]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 载入ImageNet预训练图像分类模型

In [6]:
model = models.resnet18(pretrained=True)
model = model.eval().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 87.5MB/s]


## 创建一个测试张量

In [7]:
x = torch.randn(1, 3, 256, 256).to(device)

## 进行推理预测，获得1000个类别的预测结果

In [8]:
output = model(x)

In [9]:
output.shape

torch.Size([1, 1000])

## 模型转为ONNX格式

In [10]:
with torch.no_grad():
    torch.onnx.export(
        model,                       # 要转换的模型
        x,                           # 模型的任意一组输入
        'resnet18_imagenet.onnx',    # 导出的 ONNX 文件名
        opset_version=11,            # ONNX 算子集版本，可根据模型所用模块选用不同版本的算子集
        input_names=['input'],
        output_names=['output']
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



# 水果30类图像分类模型转ONNX



## 库

In [11]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 获取模型权重文件

In [12]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth -P checkpoint

--2023-08-08 07:47:18--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth
Resolving zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)... 121.36.235.132
Connecting to zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)|121.36.235.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44854477 (43M) [binary/octet-stream]
Saving to: ‘checkpoint/fruit30_pytorch_20220814.pth’

fruit30_pytorch_202 100%[===================>]  42.78M  13.2MB/s    in 4.6s    

2023-08-08 07:47:25 (9.22 MB/s) - ‘checkpoint/fruit30_pytorch_20220814.pth’ saved [44854477/44854477]



## 导入训练好的模型

In [13]:
model = torch.load('checkpoint/fruit30_pytorch_20220814.pth')
model = model.eval().to(device)

## 创建一个测试张量

In [14]:
x = torch.randn(1, 3, 256, 256).to(device)

## 模型转为ONNX格式

In [15]:
x = torch.randn(1, 3, 256, 256).to(device)

with torch.no_grad():
    torch.onnx.export(
        model,                   # 要转换的模型
        x,                       # 模型的任意一组输入
        'resnet18_fruit30.onnx', # 导出的 ONNX 文件名
        opset_version=11,        # ONNX 算子集版本
        input_names=['input'],
        output_names=['output']
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

